In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from ephemerista import UT1Provider
from ephemerista.analysis.link_budget import LinkBudget
from ephemerista.angles import Angle
from ephemerista.assets import Asset, GroundStation, Spacecraft
from ephemerista.comms.antennas import FieldOfRegard, SimpleAntenna
from ephemerista.comms.channels import Channel
from ephemerista.comms.receiver import SimpleReceiver
from ephemerista.comms.systems import CommunicationSystem
from ephemerista.comms.transmitter import Transmitter
from ephemerista.propagators.sgp4 import SGP4
from ephemerista.scenarios import Scenario
from ephemerista.time import TimeDelta

In [ ]:
provider = UT1Provider("../tests/resources/finals2000A.all.csv")

In [ ]:
uplink = Channel(link_type="uplink", modulation="BPSK", data_rate=430502, required_eb_n0=2.3, margin=3)
downlink = Channel(link_type="downlink", modulation="BPSK", data_rate=861004, required_eb_n0=4.2, margin=3)

In [ ]:
# S-Band
frequency = 2308e6  # Hz

In [ ]:
gs_antenna = SimpleAntenna(
    gain_db=30, beamwidth_deg=5, field_of_regard=FieldOfRegard(degrees=90), design_frequency=frequency
)
gs_transmitter = Transmitter(power=4, frequency=frequency, line_loss=1.0)
gs_receiver = SimpleReceiver(system_noise_temperature=889, frequency=frequency)
gs_system = CommunicationSystem(
    channels=[uplink.channel_id, downlink.channel_id],
    transmitter=gs_transmitter,
    receiver=gs_receiver,
    antenna=gs_antenna,
)

In [ ]:
sc_antenna = SimpleAntenna(gain_db=6.5, beamwidth_deg=60, design_frequency=frequency)
sc_transmitter = Transmitter(power=1.348, frequency=frequency, line_loss=1.0)
sc_receiver = SimpleReceiver(system_noise_temperature=429, frequency=frequency)
sc_system = CommunicationSystem(
    channels=[uplink.channel_id, downlink.channel_id],
    transmitter=sc_transmitter,
    receiver=sc_receiver,
    antenna=sc_antenna,
)

In [ ]:
station_coordinates = [
    (38.017, 23.731),
    (36.971, 22.141),
    (35.501, 24.011),
    (39.326, -82.101),
    (50.750, 6.211),
]

stations = [
    Asset(
        model=GroundStation.from_lla(longitude, latitude, minimum_elevation=Angle.from_degrees(10)),
        name=f"Station {i}",
        comms=[gs_system],
    )
    for i, (latitude, longitude) in enumerate(station_coordinates)
]

In [ ]:
tle = """
1 99878U 14900A   24103.76319466  .00000000  00000-0 -11394-2 0    01
2 99878  97.5138 156.7457 0016734 205.2381 161.2435 15.13998005    06
"""

propagator = SGP4(tle=tle)
sc = Asset(model=Spacecraft(propagator=propagator), name="PHASMA", comms=[sc_system])

In [ ]:
start_date = propagator.time
end_date = start_date + TimeDelta.from_days(1)

scenario = Scenario(
    assets=[*stations, sc],
    channels=[uplink, downlink],
    name="PHASMA Link Budget",
    start_date=start_date,
    end_date=end_date,
)

In [ ]:
lb = LinkBudget(scenario=scenario)
results = lb.analyze(provider)
results.to_dataframe(stations[0], sc)

In [ ]:
results[stations[0], sc][0].plot()